In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from hmmlearn.hmm import CategoricalHMM

# --- Initial Parameters ---
start_prob = np.array([1.0, 0.0])  # Initial state probabilities (π)
trans_mat = np.array([  # Transition matrix (A)
    [0.1, 0.9],
    [0.5, 0.5]
])
emission_prob = np.array([  # Emission matrix (B)
    [0.3, 0.5, 0.2],
    [0.6, 0.2, 0.2]
])

# Define states and observations
states = ["s1", "s2"]
observations = ["v1", "v2", "v3"]

# --- Create and Configure HMM ---
# Observation sequence in numeric form (e.g., [v3, v2, v1] -> [2, 1, 0])
observed_sequence_numeric = np.array([[2], [1], [0]])

# Number of states and features (possible observations)
n_states = len(states)
n_features = len(observations)

# Create and configure the hmmlearn model
model = CategoricalHMM(n_components=n_states, n_features=n_features, n_iter=0, verbose=False)

# Manually set the parameters after model creation
model.startprob_ = start_prob
model.transmat_ = trans_mat
model.emissionprob_ = emission_prob

# --- 1. Forward Algorithm (Likelihood P(O|λ)) ---
log_likelihood = model.score(observed_sequence_numeric)
likelihood = np.exp(log_likelihood)

# --- 2. Viterbi Algorithm (Most Likely Path) ---
logprob_viterbi, best_path_indices_viterbi = model.decode(observed_sequence_numeric, algorithm="viterbi")
best_path_prob_viterbi = np.exp(logprob_viterbi)
best_path_labels_viterbi = [states[i] for i in best_path_indices_viterbi]

# --- 3. MAP Algorithm (Most Likely State Sequence) ---
logprob_map, best_path_indices_map = model.decode(observed_sequence_numeric, algorithm="map")
best_path_prob_map = np.exp(logprob_map)
best_path_labels_map = [states[i] for i in best_path_indices_map]

# --- 4. Forward-Backward Algorithm (Posterior Probabilities) ---
posterior_probs = model.predict_proba(observed_sequence_numeric)

# --- Display Results ---
print("--- HMM Model Parameters ---")
print(f"Start Probabilities (π): {model.startprob_}")
print(f"Transition Matrix (A):\n{model.transmat_}")
print(f"Emission Matrix (B):\n{model.emissionprob_}")
print(f"Observed Sequence (Numeric): {observed_sequence_numeric.flatten()}")
print("-" * 30)

# --- 1. Forward Algorithm (Likelihood P(O|λ)) ---
print(f"--- Forward Algorithm (Likelihood P(O|λ)) ---")
print(f"Log Likelihood: {log_likelihood:.6f}")
print(f"Likelihood: {likelihood:.6f}")
print(f"(This is the probability of observing the sequence {observations[2]}, {observations[1]}, {observations[0]} given the model)")
print("-" * 30)

# --- 2. Viterbi Algorithm (Most Likely State Sequence) ---
print(f"--- Viterbi Algorithm (Most Likely State Sequence) ---")
print(f"Most Likely Hidden State Sequence (Viterbi): {best_path_labels_viterbi} (Numeric: {best_path_indices_viterbi})")
print(f"Log Probability of this path: {logprob_viterbi:.6f}")
print(f"Probability of this path: {best_path_prob_viterbi:.6f}")
print("-" * 30)

# --- 3. MAP Algorithm (Most Likely State Sequence) ---
print(f"--- MAP Algorithm (Most Likely State Sequence) ---")
print(f"Most Likely Hidden State Sequence (MAP): {best_path_labels_map} (Numeric: {best_path_indices_map})")
print(f"Log Probability of this path: {logprob_map:.6f}")
print(f"Probability of this path: {best_path_prob_map:.6f}")
print("-" * 30)

# --- 4. Forward-Backward Algorithm (Posterior Probabilities) ---
print(f"--- Forward-Backward Algorithm (Posterior Probabilities P(state_t|O,λ)) ---")
print("Rows = Time Steps (for observations v3, v2, v1)")
print("Columns = States [s1, s2]")
print(posterior_probs.round(4))
print("-" * 30)

# --- HMM Structure Visualization ---

# Create directed graph for visualization
G = nx.DiGraph()
for i, state in enumerate(states):
    G.add_node(state)
for obs in observations:
    G.add_node(obs)
G.add_edge('Start', states[0], weight=start_prob[0])
G.add_edge('Start', states[1], weight=start_prob[1])

# Add transitions and emissions
for i, state_from in enumerate(states):
    for j, state_to in enumerate(states):
        if trans_mat[i, j] > 0:
            G.add_edge(state_from, state_to, weight=trans_mat[i, j])
    for j, obs in enumerate(observations):
        if emission_prob[i, j] > 0:
            G.add_edge(state_from, obs, weight=emission_prob[i, j])

# Define node positions
pos = {
    'Start': (0.5, 1.1), 's1': (0.25, 0.5), 's2': (0.75, 0.5),
    'v1': (0.1, 0.1), 'v2': (0.5, 0.1), 'v3': (0.9, 0.1)
}

# Create the plot
plt.figure(figsize=(14, 12))
nx.draw_networkx_nodes(G, pos, node_size=2500, node_color='skyblue', alpha=0.6)
nx.draw_networkx_labels(G, pos, font_size=12, font_weight='bold')
nx.draw_networkx_edges(G, pos, width=2, alpha=0.6, edge_color='gray')
nx.draw_networkx_edge_labels(G, pos, edge_labels=nx.get_edge_attributes(G, 'weight'), font_size=10)

# Add title and display
plt.title('HMM Structure Visualization', fontsize=16)
plt.axis('off')
plt.show()


Okay, let's break down this Python code step by step.

**Overall Purpose:**

This code demonstrates the fundamental concepts and algorithms associated with a Hidden Markov Model (HMM). It defines a specific HMM, provides an observed sequence of outputs, and then uses the `hmmlearn` library to:

1.  Calculate the **likelihood** of the observed sequence given the model (Forward Algorithm).
2.  Find the **most probable sequence of hidden states** that generated the observation sequence (Viterbi Algorithm).
3.  Find the sequence composed of the **most likely state at each individual time step** (MAP Algorithm, based on Forward-Backward).
4.  Calculate the **probability of being in each hidden state at each time step**, given the entire observation sequence (Forward-Backward Algorithm).
5.  **Visualize** the structure of the defined HMM.

**Code Breakdown:**

1.  **Imports:**
    *   `numpy as np`: Used for numerical operations, especially creating and manipulating arrays (matrices and vectors) for the HMM parameters.
    *   `matplotlib.pyplot as plt`: Used for plotting, specifically for visualizing the HMM structure.
    *   `networkx as nx`: A library for creating, manipulating, and studying complex networks (graphs). Used here to build the HMM visualization.
    *   `from hmmlearn.hmm import CategoricalHMM`: Imports the specific HMM class from the `hmmlearn` library suitable for discrete (categorical) observations.

2.  **Initial Parameters:**
    *   `start_prob (π)`: A NumPy array representing the initial state probabilities. `[1.0, 0.0]` means the HMM is certain to start in the first state (`s1`).
    *   `trans_mat (A)`: A 2D NumPy array representing the transition matrix. `trans_mat[i, j]` is the probability of transitioning *from* state `i` *to* state `j`.
        *   `[0.1, 0.9]`: From state `s1`, there's a 10% chance of staying in `s1` and a 90% chance of moving to `s2`.
        *   `[0.5, 0.5]`: From state `s2`, there's a 50% chance of moving to `s1` and a 50% chance of staying in `s2`.
    *   `emission_prob (B)`: A 2D NumPy array representing the emission matrix. `emission_prob[i, j]` is the probability of observing the `j`-th observation *while* in state `i`.
        *   `[0.3, 0.5, 0.2]`: In state `s1`, P(observe `v1`)=0.3, P(observe `v2`)=0.5, P(observe `v3`)=0.2.
        *   `[0.6, 0.2, 0.2]`: In state `s2`, P(observe `v1`)=0.6, P(observe `v2`)=0.2, P(observe `v3`)=0.2.

3.  **Define States and Observations:**
    *   `states`: A list of strings naming the hidden states (`s1`, `s2`).
    *   `observations`: A list of strings naming the possible observations (`v1`, `v2`, `v3`).

4.  **Create and Configure HMM:**
    *   `observed_sequence_numeric`: The sequence of observations the model will analyze (`[v3, v2, v1]`). It's converted to numeric indices (`[2, 1, 0]`) because `hmmlearn` works with numerical data. Each number corresponds to the index of the observation in the `observations` list. It's reshaped into a 2D array (`[[2], [1], [0]]`) as `hmmlearn` expects sequences as column vectors (or matrices where each row is a time step).
    *   `n_states`, `n_features`: Calculated from the lengths of the `states` and `observations` lists. These are required parameters for the `CategoricalHMM`.
    *   `model = CategoricalHMM(...)`: Creates an instance of the `CategoricalHMM` class.
        *   `n_components=n_states`: Specifies the number of hidden states.
        *   `n_features=n_features`: Specifies the number of unique possible observations.
        *   `n_iter=0`: **Crucially**, this prevents the model from trying to *learn* or *estimate* the parameters from data. We want to use the exact parameters defined earlier.
        *   `verbose=False`: Suppresses detailed output during potential (but disabled) fitting.
    *   `model.startprob_ = start_prob`, `model.transmat_ = trans_mat`, `model.emissionprob_ = emission_prob`: Manually setting the model's parameters to the predefined `start_prob`, `trans_mat`, and `emission_prob`. The trailing underscore (`_`) is a convention in `hmmlearn` (and `scikit-learn`) for attributes learned or set after initialization.

5.  **1. Forward Algorithm (Likelihood P(O|λ)):**
    *   `log_likelihood = model.score(observed_sequence_numeric)`: This method calculates the log-likelihood of the `observed_sequence_numeric` given the HMM model (`λ`, which represents the set of parameters π, A, B). Internally, this uses the Forward Algorithm. Working with log-likelihoods prevents numerical underflow issues with very small probabilities.
    *   `likelihood = np.exp(log_likelihood)`: Converts the log-likelihood back to the actual probability. This value represents P(observations | model).

6.  **2. Viterbi Algorithm (Most Likely Path):**
    *   `logprob_viterbi, best_path_indices_viterbi = model.decode(observed_sequence_numeric, algorithm="viterbi")`: This method finds the single most likely sequence of hidden states that could have generated the observed sequence.
        *   `algorithm="viterbi"`: Specifies the Viterbi algorithm.
        *   `logprob_viterbi`: The log probability of this single most likely path.
        *   `best_path_indices_viterbi`: A NumPy array containing the indices (0 or 1) of the states in the most likely path.
    *   `best_path_prob_viterbi = np.exp(logprob_viterbi)`: Converts the log probability of the path to the actual probability.
    *   `best_path_labels_viterbi = [states[i] for i in best_path_indices_viterbi]`: Converts the numeric state indices back to their string labels (`s1`, `s2`).

7.  **3. MAP Algorithm (Most Likely State Sequence):**
    *   `logprob_map, best_path_indices_map = model.decode(observed_sequence_numeric, algorithm="map")`: This method finds the sequence of states where each state in the sequence is the *individually* most likely state at that specific time step, given the *entire* observation sequence. This is achieved by running the Forward-Backward algorithm first to calculate posterior probabilities (see step 8) and then picking the state with the highest posterior probability at each time step.
        *   `algorithm="map"`: Specifies the Maximum A Posteriori decoding based on individual state posteriors. **Note:** This path is *not guaranteed* to be the same as the Viterbi path, although it often is. Viterbi finds the globally best *sequence*, while MAP finds the sequence of locally best *states*.
        *   `logprob_map`: The log probability associated with the MAP path (calculated differently than Viterbi's path probability).
        *   `best_path_indices_map`: A NumPy array containing the indices of the states in the MAP path.
    *   `best_path_prob_map = np.exp(logprob_map)`: Converts the log probability.
    *   `best_path_labels_map = [states[i] for i in best_path_indices_map]`: Converts indices to labels.

8.  **4. Forward-Backward Algorithm (Posterior Probabilities):**
    *   `posterior_probs = model.predict_proba(observed_sequence_numeric)`: This method calculates the posterior probabilities using the Forward-Backward algorithm.
    *   The result `posterior_probs` is a 2D NumPy array where `posterior_probs[t, i]` gives the probability of being in state `i` at time step `t`, given the *entire* observation sequence `O` and the model parameters `λ`. Mathematically, this is P(state `i` at time `t` | `O`, `λ`).

9.  **Display Results:**
    *   This section uses `print()` statements with f-strings to clearly output:
        *   The HMM parameters that were set.
        *   The numeric observed sequence.
        *   The results (log-likelihood and likelihood) from the Forward algorithm.
        *   The results (most likely path labels/indices, log probability, probability) from the Viterbi algorithm.
        *   The results (most likely path labels/indices, log probability, probability) from the MAP algorithm.
        *   The posterior probabilities calculated by the Forward-Backward algorithm, formatted for readability.

10. **HMM Structure Visualization:**
    *   `G = nx.DiGraph()`: Creates an empty directed graph object using `networkx`.
    *   `G.add_node(...)`: Adds nodes to the graph representing the hidden states (`s1`, `s2`) and the possible observations (`v1`, `v2`, `v3`). A special 'Start' node is added for visualizing initial probabilities.
    *   `G.add_edge(...)`: Adds directed edges to represent:
        *   Initial probabilities (from 'Start' to `s1`/`s2`).
        *   Transitions between states (`s1` to `s1`, `s1` to `s2`, etc.).
        *   Emissions from states to observations (`s1` to `v1`, `s1` to `v2`, etc.).
        *   The `weight` attribute of each edge stores the corresponding probability (start, transition, or emission). Edges with probability 0 are omitted.
    *   `pos = {...}`: Manually defines the (x, y) coordinates for positioning each node in the plot for a clear layout.
    *   `plt.figure(...)`: Creates a Matplotlib figure to draw on.
    *   `nx.draw_networkx_nodes(...)`, `nx.draw_networkx_labels(...)`, `nx.draw_networkx_edges(...)`, `nx.draw_networkx_edge_labels(...)`: `networkx` functions (interfacing with Matplotlib) to draw the nodes, labels, edges, and edge weights (probabilities) using the defined positions.
    *   `plt.title(...)`, `plt.axis('off')`, `plt.show()`: Standard Matplotlib commands to add a title, hide the axes, and display the plot.

In summary, the code sets up a known HMM, runs standard HMM algorithms on a given observation sequence using `hmmlearn`, prints the results of these algorithms, and provides a visual representation of the HMM's structure and parameters.

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

# Define states and observations
states = ["s1", "s2"]
observations = ["v1", "v2", "v3"]

# Create a directed graph
G = nx.DiGraph()

# Add state and observation nodes
G.add_nodes_from(states + observations)

# Define transitions (from state to state) and emissions (from state to observation) with probabilities
transitions = [("s1", "s1", 0.1), ("s1", "s2", 0.9), 
               ("s2", "s1", 0.5), ("s2", "s2", 0.5)]
emissions = [("s1", "v1", 0.3), ("s1", "v2", 0.5), ("s1", "v3", 0.2),
             ("s2", "v1", 0.6), ("s2", "v2", 0.2), ("s2", "v3", 0.2)]

# Add edges with weights (probabilities)
G.add_weighted_edges_from(transitions)
G.add_weighted_edges_from(emissions)

# Set node colors: states in blue, observations in green
node_colors = ['lightblue' if node in states else 'lightgreen' for node in G.nodes]

# Set position using spring layout (to avoid overlap)
pos = nx.spring_layout(G, seed=42, k=1.5)

# Plot the graph
plt.figure(figsize=(10, 8))
nx.draw(G, pos, with_labels=True, node_size=3000, node_color=node_colors, font_size=12, font_weight='bold', arrows=True)

# Draw edge labels with probabilities
edge_labels = nx.get_edge_attributes(G, 'weight')
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=10)

# Title and show
plt.title('Hidden Markov Model with Probabilities')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

# Define states and observations
states = ["s1", "s2"]
observations = ["v1", "v2", "v3"]

# Create a directed graph
G = nx.DiGraph()

# Add state and observation nodes
G.add_nodes_from(states + observations)

# Define initial probabilities, transitions (state to state) and emissions (state to observation) with probabilities
start_prob = {"s1": 1.0, "s2": 0.0}  # Initial state probabilities
transitions = [("s1", "s1", 0.1), ("s1", "s2", 0.9), 
               ("s2", "s1", 0.5), ("s2", "s2", 0.5)]
emissions = [("s1", "v1", 0.3), ("s1", "v2", 0.5), ("s1", "v3", 0.2),
             ("s2", "v1", 0.6), ("s2", "v2", 0.2), ("s2", "v3", 0.2)]

# Add edges with weights (probabilities)
G.add_weighted_edges_from(transitions)
G.add_weighted_edges_from(emissions)

# Set node colors: states in blue, observations in green
node_colors = ['lightblue' if node in states else 'lightgreen' for node in G.nodes]

# Set position using spring layout (to avoid overlap)
pos = nx.spring_layout(G, seed=42, k=1.5)

# Plot the graph
plt.figure(figsize=(12, 10))

# Draw the graph (states are in blue, observations in green)
nx.draw(G, pos, with_labels=True, node_size=3000, node_color=node_colors, font_size=12, font_weight='bold', arrows=True)

# Draw edge labels with probabilities
edge_labels = nx.get_edge_attributes(G, 'weight')
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=10)

# Draw initial probability annotations
start_state_label = f"Start: s1 = {start_prob['s1']}, s2 = {start_prob['s2']}"
plt.text(0.5, 1.1, start_state_label, horizontalalignment='center', fontsize=12, color='black', weight='bold')

# Add transition and emission annotations for clarity
for (start, end, prob) in transitions:
    plt.text(pos[start][0], pos[start][1] + 0.1, f"p={prob:.2f}", fontsize=10, color='blue')

for (state, obs, prob) in emissions:
    plt.text(pos[state][0], pos[state][1] - 0.1, f"p={prob:.2f}", fontsize=10, color='green')

# Title and show
plt.title('Hidden Markov Model (HMM) Representation with Probabilities')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from hmmlearn.hmm import CategoricalHMM

# Initial Parameters
start_prob = np.array([1.0, 0.0])  # Initial state probabilities (π)
trans_mat = np.array([  # Transition matrix (A)
    [0.1, 0.9],
    [0.5, 0.5]
])
emission_prob = np.array([  # Emission matrix (B)
    [0.3, 0.5, 0.2],
    [0.6, 0.2, 0.2]
])

# Observation sequence in numeric form (e.g., [v3, v2, v1] -> [2, 1, 0])
observed_sequence_numeric = np.array([[2], [1], [0]])

# Define states and observations for clarity
states = ["s1", "s2"]  # State labels
observations = ["v1", "v2", "v3"]  # Observation labels

# Create and configure the hmmlearn model
model = CategoricalHMM(n_components=2, n_features=3, n_iter=0, verbose=False)
model.startprob_ = start_prob
model.transmat_ = trans_mat
model.emissionprob_ = emission_prob

# Create directed graph for the HMM visualization
G = nx.DiGraph()

# Add state nodes
for i, state in enumerate(states):
    G.add_node(state, label=state)

# Add observation nodes
for observation in observations:
    G.add_node(observation, label=observation)

# Add transition edges with probabilities (including self-transitions)
for i, state_from in enumerate(states):
    for j, state_to in enumerate(states):
        trans_prob = trans_mat[i, j]
        if trans_prob > 0:  # Only add edges with non-zero transition probability
            G.add_edge(state_from, state_to, weight=trans_prob)

# Add emission edges (representing each state emitting observations with their probabilities)
for i, state in enumerate(states):
    for j, observation in enumerate(observations):
        emit_prob = emission_prob[i, j]
        if emit_prob > 0:  # Only add edges with non-zero emission probability
            G.add_edge(state, observation, weight=emit_prob)

# Add the start probability edge from a virtual "Start" node to the initial state
G.add_edge('Start', states[0], weight=start_prob[0])
G.add_edge('Start', states[1], weight=start_prob[1])

# Prepare edge labels (for transition and emission probabilities)
labels = nx.get_edge_attributes(G, 'weight')

# Define custom positions to improve alignment
pos = {
    'Start': (0.5, 1.1),  # Place 'Start' at the top
    's1': (0.25, 0.5),    # State 1 to the left
    's2': (0.75, 0.5),    # State 2 to the right
    'v1': (0.1, 0.1),     # Observation v1 below state 1
    'v2': (0.5, 0.1),     # Observation v2 centered below the states
    'v3': (0.9, 0.1)      # Observation v3 below state 2
}

# Adjust position of nodes
for i, state in enumerate(states):
    pos[state] = (pos[state][0], pos[state][1])  # Slightly adjust y for better vertical alignment

# Create the plot
plt.figure(figsize=(14, 12))

# Draw nodes
nx.draw_networkx_nodes(G, pos, node_size=2500, node_color='skyblue', alpha=0.6)

# Draw state labels
nx.draw_networkx_labels(G, pos, font_size=12, font_weight='bold', font_color='black')

# Draw edges
nx.draw_networkx_edges(G, pos, width=2, alpha=0.6, edge_color='gray')

# Draw edge labels for probabilities
nx.draw_networkx_edge_labels(G, pos, edge_labels=labels, font_size=10)

# Add title
plt.title(f'HMM Structure Visualization (Transitions and Emissions)', fontsize=16)
plt.axis('off')  # Hide axes

# Show the plot
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from hmmlearn.hmm import CategoricalHMM

# Initial Parameters
start_prob = np.array([1.0, 0.0])  # Initial state probabilities (π)
trans_mat = np.array([  # Transition matrix (A)
    [0.1, 0.9],
    [0.5, 0.5]
])
emission_prob = np.array([  # Emission matrix (B)
    [0.3, 0.5, 0.2],
    [0.6, 0.2, 0.2]
])

# Define states and observations
states = ["s1", "s2"]
observations = ["v1", "v2", "v3"]

# Create HMM model and assign parameters
model = CategoricalHMM(n_components=2, n_features=3, n_iter=0)
model.startprob_ = start_prob
model.transmat_ = trans_mat
model.emissionprob_ = emission_prob

# Create directed graph for visualization
G = nx.DiGraph()
for i, state in enumerate(states):
    G.add_node(state)
for obs in observations:
    G.add_node(obs)
G.add_edge('Start', states[0], weight=start_prob[0])
G.add_edge('Start', states[1], weight=start_prob[1])

# Add transitions and emissions
for i, state_from in enumerate(states):
    for j, state_to in enumerate(states):
        if trans_mat[i, j] > 0:
            G.add_edge(state_from, state_to, weight=trans_mat[i, j])
    for j, obs in enumerate(observations):
        if emission_prob[i, j] > 0:
            G.add_edge(state_from, obs, weight=emission_prob[i, j])

# Define node positions
pos = {
    'Start': (0.5, 1.1), 's1': (0.25, 0.5), 's2': (0.75, 0.5),
    'v1': (0.1, 0.1), 'v2': (0.5, 0.1), 'v3': (0.9, 0.1)
}

# Create the plot
plt.figure(figsize=(14, 12))
nx.draw_networkx_nodes(G, pos, node_size=2500, node_color='skyblue', alpha=0.6)
nx.draw_networkx_labels(G, pos, font_size=12, font_weight='bold')
nx.draw_networkx_edges(G, pos, width=2, alpha=0.6, edge_color='gray')
nx.draw_networkx_edge_labels(G, pos, edge_labels=nx.get_edge_attributes(G, 'weight'), font_size=10)

# Add title and display
plt.title('HMM Structure Visualization', fontsize=16)
plt.axis('off')
plt.show()


In [ ]:
import numpy as np
from hmmlearn.hmm import CategoricalHMM

# Initial Parameters
start_prob = np.array([1.0, 0.0])  # Initial state probabilities (π)
trans_mat = np.array([  # Transition matrix (A)
    [0.1, 0.9],
    [0.5, 0.5]
])
emission_prob = np.array([  # Emission matrix (B)
    [0.3, 0.5, 0.2],
    [0.6, 0.2, 0.2]
])

# Observation sequence in numeric form (e.g., [v3, v2, v1] -> [2, 1, 0])
observed_sequence_numeric = np.array([[2], [1], [0]])

# Define states and observations for clarity
states = ["s1", "s2"]  # State labels
observations = ["v1", "v2", "v3"]  # Observation labels

# Number of states and features (possible observations)
n_states = len(states)
n_features = len(observations)

# Create and configure the hmmlearn model
model = CategoricalHMM(n_components=n_states, n_features=n_features, n_iter=0, verbose=False)

# Manually set the parameters after model creation
model.startprob_ = start_prob
model.transmat_ = trans_mat
model.emissionprob_ = emission_prob

# Print model parameters
print("--- HMM Model Parameters ---")
print(f"Start Probabilities (π): {model.startprob_}")
print(f"Transition Matrix (A):\n{model.transmat_}")
print(f"Emission Matrix (B):\n{model.emissionprob_}")
print(f"Observed Sequence (Numeric): {observed_sequence_numeric.flatten()}")
print("-" * 30)

# --- 1. Forward Algorithm (Likelihood P(O|λ)) ---
log_likelihood = model.score(observed_sequence_numeric)
likelihood = np.exp(log_likelihood)

print(f"--- Forward Algorithm (Likelihood P(O|λ)) ---")
print(f"Log Likelihood: {log_likelihood:.6f}")
print(f"Likelihood: {likelihood:.6f}")
print(f"(This is the probability of observing the sequence {observations[2]}, {observations[1]}, {observations[0]} given the model)")
print("-" * 30)

# --- 2. Viterbi Algorithm (Most Likely Path) ---
logprob_viterbi, best_path_indices_viterbi = model.decode(observed_sequence_numeric, algorithm="viterbi")
best_path_prob_viterbi = np.exp(logprob_viterbi)
best_path_labels_viterbi = [states[i] for i in best_path_indices_viterbi]

print(f"--- Viterbi Algorithm (Most Likely State Sequence) ---")
print(f"Most Likely Hidden State Sequence (Viterbi): {best_path_labels_viterbi} (Numeric: {best_path_indices_viterbi})")
print(f"Log Probability of this path: {logprob_viterbi:.6f}")
print(f"Probability of this path: {best_path_prob_viterbi:.6f}")
print("-" * 30)

# --- 3. MAP Algorithm (Most Likely State Sequence) ---
logprob_map, best_path_indices_map = model.decode(observed_sequence_numeric, algorithm="map")
best_path_prob_map = np.exp(logprob_map)
best_path_labels_map = [states[i] for i in best_path_indices_map]

print(f"--- MAP Algorithm (Most Likely State Sequence) ---")
print(f"Most Likely Hidden State Sequence (MAP): {best_path_labels_map} (Numeric: {best_path_indices_map})")
print(f"Log Probability of this path: {logprob_map:.6f}")
print(f"Probability of this path: {best_path_prob_map:.6f}")
print("-" * 30)

# --- 4. Forward-Backward Algorithm (Posterior Probabilities) ---
posterior_probs = model.predict_proba(observed_sequence_numeric)

print(f"--- Forward-Backward Algorithm (Posterior Probabilities P(state_t|O,λ)) ---")
print("Rows = Time Steps (for observations v3, v2, v1)")
print("Columns = States [s1, s2]")
print(posterior_probs.round(4))
print("-" * 30)


In [ ]:
CategoricalHMM?

In [ ]:
import numpy as np
from hmmlearn.hmm import CategoricalHMM

start_prob = np.array([1.0, 0.0])

trans_mat = np.array([
    [0.1, 0.9],
    [0.5, 0.5]
])

emission_prob = np.array([
    [0.3, 0.5, 0.2],
    [0.6, 0.2, 0.2]
])

observed_sequence_numeric = np.array([[2], [1], [0]])

states = ["s1", "s2"]
observations = ["v1", "v2", "v3"]

model = CategoricalHMM(n_components=len(states), n_iter=0, verbose=False)

model.startprob_ = start_prob
model.transmat_ = trans_mat
model.emissionprob_ = emission_prob

print("--- HMM Model Parameters ---")
print(f"Start Probabilities (π): {model.startprob_}")
print(f"Transition Matrix (A):\n{model.transmat_}")
print(f"Emission Matrix (B):\n{model.emissionprob_}")
print(f"Observed Sequence (Numeric): {observed_sequence_numeric.flatten()}")
print("-" * 30)

log_likelihood = model.score(observed_sequence_numeric)
likelihood = np.exp(log_likelihood)

print(f"--- Forward Algorithm (Likelihood P(O|λ)) ---")
print(f"Log Likelihood: {log_likelihood:.6f}")
print(f"Likelihood: {likelihood:.6f}")
print(f"(This is the probability of observing the sequence {observations[2]}, {observations[1]}, {observations[0]} given the model)")
print("-" * 30)

logprob_viterbi, best_path_indices = model.decode(observed_sequence_numeric, algorithm="viterbi")
best_path_prob = np.exp(logprob_viterbi)
best_path_labels = [states[i] for i in best_path_indices]

print(f"--- Viterbi Algorithm (Most Likely State Sequence) ---")
print(f"Most Likely Hidden State Sequence: {best_path_labels} (Numeric: {best_path_indices})")
print(f"Log Probability of this path: {logprob_viterbi:.6f}")
print(f"Probability of this path: {best_path_prob:.6f}")
print("-" * 30)

posterior_probs = model.predict_proba(observed_sequence_numeric)

print(f"--- Forward-Backward Algorithm (Posterior Probabilities P(state_t|O,λ)) ---")
print("Rows = Time Steps (for observations v3, v2, v1)")
print("Columns = States [s1, s2]")
print(posterior_probs.round(4))
print("-" * 30)


In [ ]:
import numpy as np
from hmmlearn.hmm import CategoricalHMM

# --- Parameters ---
# Define the model parameters. These include:
# - Initial state probabilities (π)
# - Transition matrix (A)
# - Emission matrix (B)

# Initial state probabilities (π)
# This represents the initial distribution of the states. In our case, 
# we start with state s1 with a 100% probability and never start in state s2.
start_prob = np.array([1.0, 0.0])  # Starting in state s1 (prob = 1)

# Transition matrix (A)
# This matrix defines the probabilities of transitioning from one state to another.
# For example, from state s1, there's a 10% chance of staying in s1, and a 90% chance of moving to s2.
# From state s2, there's a 50% chance of moving to s1 and a 50% chance of staying in s2.
trans_mat = np.array([
    [0.1, 0.9],  # Transition probabilities from state s1: P(s1 -> s1) = 0.1, P(s1 -> s2) = 0.9
    [0.5, 0.5]   # Transition probabilities from state s2: P(s2 -> s1) = 0.5, P(s2 -> s2) = 0.5
])

# Emission matrix (B)
# This matrix defines the probability of observing each of the observations (v1, v2, v3) 
# given the current state.
# For example, when in state s1:
# P(v1|s1) = 0.3, P(v2|s1) = 0.5, P(v3|s1) = 0.2
# When in state s2:
# P(v1|s2) = 0.6, P(v2|s2) = 0.2, P(v3|s2) = 0.2
emission_prob = np.array([
    [0.3, 0.5, 0.2],  # Emission probabilities for state s1: P(v1|s1) = 0.3, P(v2|s1) = 0.5, P(v3|s1) = 0.2
    [0.6, 0.2, 0.2]   # Emission probabilities for state s2: P(v1|s2) = 0.6, P(v2|s2) = 0.2, P(v3|s2) = 0.2
])

# Observation sequence (v1=0, v2=1, v3=2)
# We define an observation sequence [v3, v2, v1] as a numeric sequence [2, 1, 0].
observed_sequence_numeric = np.array([[2], [1], [0]])  # Shape (n_samples, 1)

# State and Observation labels (for clarity)
states = ["s1", "s2"]  # Two hidden states: s1 and s2
observations = ["v1", "v2", "v3"]  # Three possible observations: v1, v2, v3

# --- Create and Configure hmmlearn Model ---
# Initialize the HMM model with 2 components (states).
# n_iter=0 means we are not training the model, we are directly assigning the parameters.
model = CategoricalHMM(n_components=len(states), n_iter=0, verbose=False)

# Set the model parameters
model.startprob_ = start_prob  # Set the initial state probabilities (π)
model.transmat_ = trans_mat    # Set the transition matrix (A)
model.emissionprob_ = emission_prob  # Set the emission matrix (B)
print(model.startprob_,model.transmat_,model.emissionprob_,sep='\n')
# --- Print Model Parameters ---
print("--- HMM Model Parameters ---")
print(f"Start Probabilities (π): {model.startprob_}")
print(f"Transition Matrix (A):\n{model.transmat_}")
print(f"Emission Matrix (B):\n{model.emissionprob_}")
print(f"Observed Sequence (Numeric): {observed_sequence_numeric.flatten()}")
print("-" * 30)

# --- 1. Forward Algorithm (Likelihood P(O|λ)) ---
# The Forward algorithm calculates the likelihood of observing the sequence of events (O)
# given the model parameters (λ). It is internally implemented in model.score().
log_likelihood = model.score(observed_sequence_numeric)  # Log-likelihood of the observation sequence
likelihood = np.exp(log_likelihood)  # Convert log-likelihood to probability

# Print the result of the Forward Algorithm
print(f"--- Forward Algorithm (Likelihood P(O|λ)) ---")
print(f"Log Likelihood: {log_likelihood:.6f}")
print(f"Likelihood: {likelihood:.6f}")
print(f"(This is the probability of observing the sequence {observations[2]}, {observations[1]}, {observations[0]} given the model)")
print("-" * 30)

# --- 2. Viterbi Algorithm (Most Likely State Sequence) ---
# The Viterbi algorithm is used to find the most likely sequence of hidden states 
# that could have generated the observed sequence.
# The model.decode() function applies the Viterbi algorithm.
logprob_viterbi, best_path_indices = model.decode(observed_sequence_numeric, algorithm="viterbi")
best_path_prob = np.exp(logprob_viterbi)  # Convert log-probability to normal probability
best_path_labels = [states[i] for i in best_path_indices]  # Convert numeric indices to state labels

# Print the result of the Viterbi Algorithm
print(f"--- Viterbi Algorithm (Most Likely State Sequence) ---")
print(f"Most Likely Hidden State Sequence: {best_path_labels} (Numeric: {best_path_indices})")
print(f"Log Probability of this path: {logprob_viterbi:.6f}")
print(f"Probability of this path: {best_path_prob:.6f}")
print("-" * 30)

# --- 3. Forward-Backward Algorithm (Posterior Probabilities) ---
# The Forward-Backward algorithm calculates the posterior probabilities of the hidden states
# at each time step, given the entire observed sequence. This is done using model.predict_proba().
posterior_probs = model.predict_proba(observed_sequence_numeric)

# Print the result of the Forward-Backward Algorithm
print(f"--- Forward-Backward Algorithm (Posterior Probabilities P(state_t|O,λ)) ---")
print("Rows = Time Steps (for observations v3, v2, v1)")
print("Columns = States [s1, s2]")
print(posterior_probs.round(4))  # Rounded for readability
print("-" * 30)

# You can verify that sum(posterior_probs[t,:]) == 1.0 for each time step t (since it's a probability distribution).
